In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "<YOUR-CLIENT-ID>", # Replace with actual Client ID
    "fs.azure.account.oauth2.client.secret": "<YOUR-CLIENT-SECRET>", # Replace with actual Client Secret
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<YOUR-TENANT-ID>/oauth2/token" # Replace with actual Tenant ID
}

dbutils.fs.mount(
    source = "abfss://<CONTAINER>@<STORAGE-ACCOUNT>.dfs.core.windows.net/", # Replace with actual container and storage account
    mount_point = "/mnt/Datalake",
    extra_configs = configs
)


True

In [0]:
%fs
ls "/mnt/Datalake"

path,name,size,modificationTime
dbfs:/mnt/Datalake/raw-data/,raw-data/,0,1735767941000
dbfs:/mnt/Datalake/transformed-data/,transformed-data/,0,1735767954000


In [0]:
spark

In [0]:
athletes_df = spark.read.csv("/mnt/Datalake/raw-data/athletes.csv", header=True, inferSchema=True)
coaches_df = spark.read.csv("/mnt/Datalake/raw-data/coaches.csv", header=True, inferSchema=True)
entriesgender_df = spark.read.csv("/mnt/Datalake/raw-data/entriesgender.csv", header=True, inferSchema=True)
medals_df = spark.read.csv("/mnt/Datalake/raw-data/medals.csv", header=True, inferSchema=True)
teams_df = spark.read.csv("/mnt/Datalake/raw-data/teams.csv", header=True, inferSchema=True)


In [0]:
athletes_df.show(5)  # Display the first 5 rows

+-----------------+-------+-------------------+
|             Name|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows



In [0]:
athletes_df.printSchema()  # Display the schema of the DataFrame

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches_df.show(5)  # Display the first 5 rows

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [0]:
coaches_df.printSchema()  # Display the schema of the DataFrame

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender_df.show(5)  # Display the first 5 rows

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows



In [0]:
entriesgender_df.printSchema()  # Display the schema of the DataFrame

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals_df.show(5)  # Display the first 5 rows

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [0]:
medals_df.printSchema()  # Display the schema of the DataFrame

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams_df.show(5)  # Display the first 5 rows

+--------+--------------+--------------------+-----+
|TeamName|    Discipline|             Country|Event|
+--------+--------------+--------------------+-----+
| Belgium|3x3 Basketball|             Belgium|  Men|
|   China|3x3 Basketball|People's Republic...|  Men|
|   China|3x3 Basketball|People's Republic...|Women|
|  France|3x3 Basketball|              France|Women|
|   Italy|3x3 Basketball|               Italy|Women|
+--------+--------------+--------------------+-----+
only showing top 5 rows



In [0]:
teams_df.printSchema()  # Display the schema of the DataFrame

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries =  medals_df.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender_df.withColumn(
    'Avg_Female', entriesgender_df['Female'] / entriesgender_df['Total']
).withColumn(
    'Avg_Male', entriesgender_df['Male'] / entriesgender_df['Total']
)
average_entries_by_gender.show()
     

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
# Assuming `df` is the transformed DataFrame
target_path = "/mnt/Datalake/transformed-data/transformed_athletes_csv"  # specify your target location

# Save as CSV
athletes_df.write.csv(target_path, header=True, mode="overwrite")


In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/transformed-data/athletes")
